<a href="https://colab.research.google.com/github/meti-94/BERT-QA/blob/main/finetune_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Finetuning BERT on **ReverbSimpleQuestions** and **SimpleQuestions** dataset

In [1]:
!pip install git+https://github.com/huggingface/transformers -q
!pip uninstall -y tensorflow -q
!pip install datasets -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 27.3 MB/s 
     |████████████████████████████████| 596 kB 63.0 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
     |████████████████████████████████| 346 kB 36.5 MB/s 
     |████████████████████████████████| 86 kB 4.3 MB/s 
     |████████████████████████████████| 212 kB 76.2 MB/s 
     |████████████████████████████████| 140 kB 75.6 MB/s 
     |████████████████████████████████| 1.1 MB 30.8 MB/s 
     |████████████████████████████████| 127 kB 71.8 MB/s 
     |████████████████████████████████| 144 kB 70.4 MB/s 
     |████████████████████████████████| 271 kB 2.0 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 112 kB 77.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is th

In [2]:
!git clone https://github.com/meti-94/BERT-QA.git

Cloning into 'BERT-QA'...
remote: Enumerating objects: 531, done.
remote: Total 531 (delta 0), reused 0 (delta 0), pack-reused 531
Receiving objects: 100% (531/531), 95.82 MiB | 15.28 MiB/s, done.
Resolving deltas: 100% (235/235), done.


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
import pandas as pd
SQ_train = pd.read_csv('/content/BERT-QA/BuboQA/Freebase/train.txt', sep='\t').iloc[:, 5].to_list()
SQ_valid = pd.read_csv('/content/BERT-QA/BuboQA/Freebase/valid.txt', sep='\t').iloc[:, 5].to_list()
RSQ_train = pd.read_excel('/content/drive/MyDrive/QA/train.xlsx')['Question'].to_list()
RSQ_valid = pd.read_excel('/content/drive/MyDrive/QA/valid.xlsx')['Question'].to_list()
train_data = SQ_train+RSQ_train
valid_data = SQ_valid+RSQ_valid
with open('sq_train.txt', 'w') as fout:
  fout.write('\n'.join(SQ_train))
with open('sq_valid.txt', 'w') as fout:
  fout.write('\n'.join(SQ_valid))
with open('rsq_train.txt', 'w') as fout:
  fout.write('\n'.join(RSQ_train))
with open('rsq_valid.txt', 'w') as fout:
  fout.write('\n'.join(RSQ_valid))
with open('train_data.txt', 'w') as fout:
  fout.write('\n'.join(train_data))
with open('valid_data.txt', 'w') as fout:
  fout.write('\n'.join(valid_data))

# print(len(SQ_train), len(SQ_valid), len(RSQ_train), len(RSQ_valid), len(train_data), len(valid_data))

In [5]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 97916, done.
remote: Counting objects: 100% (467/467), done.
remote: Compressing objects: 100% (262/262), done.
remote: Total 97916 (delta 221), reused 366 (delta 180), pack-reused 97449
Receiving objects: 100% (97916/97916), 91.47 MiB | 17.68 MiB/s, done.
Resolving deltas: 100% (71930/71930), done.


In [6]:
!python /content/transformers/examples/pytorch/language-modeling/run_mlm.py \
  --model_name_or_path bert-base-uncased \
  --train_file /content/sq_train.txt \
  --validation_file /content/sq_valid.txt \
  --per_device_train_batch_size 16 \
  --per_device_eval_batch_size 16 \
  --do_train \
  --do_eval \
  --output_dir ./finetune-mlm \
  --line_by_line \
  --num_train_epochs 5 \
  --save_total_limit 2

06/10/2022 09:45:24 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
06/10/2022 09:45:24 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=N

In [7]:
!cp -r ./finetune-mlm /content/drive/MyDrive/QA/finetune/sq/